## Separation Anxiety

In [ ]:
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

from bokeh.io import output_notebook
output_notebook()


In [ ]:
from math import *  # type: ignore


class Vec:
    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z

    def dot(self, other: "Vec") -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z

    @staticmethod
    def Survey(inc: float, azi: float) -> "Vec":
        sini, cosi = sin(inc), cos(inc)
        sina, cosa = sin(azi), cos(azi)
        return Vec(sini * sina, sini * cosa, cosi)

    def normalize(self) -> float:
        n = self.norm()
        if n != 0:
            s = 1 / n
            self.x *= s
            self.y *= s
            self.z *= s
        return n

    def norm(self) -> float:
        return sqrt(self.dot(self))

    def copy(self) -> "Vec":
        return Vec(self.x, self.y, self.z)

    def __add__(self, other: "Vec") -> "Vec":
        return Vec(self.x + other.x, self.y + other.y, self.z + other.z)

    def __sub__(self, other: "Vec") -> "Vec":
        return Vec(self.x - other.x, self.y - other.y, self.z - other.z)

    def __rmul__(self, s: float) -> "Vec":
        return Vec(self.x * s, self.y * s, self.z * s)

    def __str__(self) -> str:
        return f"({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"

In [ ]:
from math import *  # type: ignore


class Ellipse:
    def __init__(self, a: float, b: float, r: float):
        self.a = a
        self.b = b
        self.set_rotation(r)
        self.o = Vec(0, 0, 0)

    def set_rotation(self, rot: float) -> None:
        self.rot = rot
        sinr = sin(rot)
        cosr = cos(rot)
        self.du = Vec(cosr, sinr, 0.0)
        self.dv = Vec(-sinr, cosr, 0.0)
        self.u = self.a * self.du
        self.v = self.b * self.dv

    def radius(self, t: float) -> float:
        sinr = sin(t)
        cosr = cos(t)
        p = Vec(cosr, sinr, 0)
        return 1 / sqrt(p.x * p.x / (self.a * self.a) + p.y * p.y / (self.b * self.b))

    def parameterized(self, t: float) -> Vec:
        sint = sin(t)
        cost = cos(t)
        return cost * self.u + sint * self.v

    def project(self, p: Vec) -> Vec:
        return Vec(p.dot(self.du), p.dot(self.dv), 0)

    def rotate(self, p: Vec) -> Vec:
        return p.x * self.du + p.y * self.dv

    def tangent(self, s: float) -> tuple[Vec, Vec]:
        r = self.radius(s)
        p = Vec(r * cos(s), r * sin(s), 0)
        t = Vec(p.y / (self.b * self.b), -p.x / (self.a * self.a), 0)
        return self.rotate(p), self.rotate(t)

    def scale(self, k: float) -> "Ellipse":
        scaled = Ellipse(self.a * k, self.b * k, self.rot)
        scaled.o = self.o
        return scaled

In [ ]:
class Cov:
    def __init__(self):
        self.ee = 0.0
        self.nn = 0.0
        self.vv = 0.0
        self.en = 0.0
        self.ev = 0.0
        self.nv = 0.0

    def add(self, vec: Vec):
        self.ee += vec.x * vec.x
        self.nn += vec.y * vec.y
        self.vv += vec.z * vec.z
        self.en += vec.x * vec.y
        self.ev += vec.x * vec.z
        self.nv += vec.y * vec.z

    def pedal(self, vec: Vec):
        x = vec.x * (vec.x * self.ee + vec.y * self.en + vec.z * self.ev)
        y = vec.y * (vec.x * self.en + vec.y * self.nn + vec.z * self.nv)
        z = vec.z * (vec.x * self.ev + vec.y * self.nv + vec.z * self.vv)
        return x + y + z

    def radius(self, vec: Vec):
        return sqrt(abs(self.pedal(vec)))

    def __str__(self) -> str:
        return f"[{self.ee:8.3f},{self.nn:8.3f},{self.vv:8.3f},{self.en:8.3f},{self.ev:8.3f},{self.nv:8.3f}]"


def lha(inc, azi):
    sini, cosi = sin(inc), cos(inc)
    sina, cosa = sin(azi), cos(azi)
    return (
        Vec(cosa, -sina, 0),
        Vec(cosi * sina, cosi * cosa, -sini),
        Vec(sini * sina, sini * cosa, cosi),
    )

In [ ]:
import numpy as np


def ellipse_to_cov(ell: Ellipse) -> Cov:
    cov = Cov()
    cov.add(ell.a * ell.du)
    cov.add(ell.b * ell.dv)
    return cov


def combined_ellipse_to_cov(e1: Ellipse, e2: Ellipse) -> Cov:
    cov = Cov()
    cov.add(e1.a * e1.du)
    cov.add(e1.b * e1.dv)
    cov.add(e2.a * e2.du)
    cov.add(e2.b * e2.dv)
    return cov


def cov_to_ellipse(cov: Cov) -> Ellipse:
    a = np.array([[cov.ee, cov.en], [cov.en, cov.nn]])
    svd = np.linalg.svd(a)
    u = svd.U[0]
    theta = atan2(u[1], u[0])
    return Ellipse(sqrt(svd.S[0]), sqrt(svd.S[1]), theta)

In [ ]:
from typing import Callable, Optional
from scipy.stats.distributions import chi2
import ipywidgets as widgets

from bokeh.plotting import figure


class Setup:
    k = 3.5


setup = Setup()


def std_rot(rot: float) -> float:
    rot += pi
    if rot == 0:
        return rot - pi
    else:
        rot = fmod(rot,  2 * pi)
        return (rot if rot > 0 else rot + 2 * pi) - pi


class Scaled:
    e1: Ellipse
    e2: Ellipse
    e3: Ellipse
    e4: Ellipse
    observer: Optional[Callable]

    mpl_palette = [
        "#1f77b4",
        "#ff7f0e",
        "#2ca02c",
        "#d62728",
        "#9467bd",
        "#8c564b",
        "#e377c2",
        "#7f7f7f",
        "#bcbd22",
        "#17becf",
    ]

    def __init__(self, ax: figure):
        self.sep = Vec(0, 0, 0)
        self.radius = 0.0
        self.k = 1.0
        self.cov = Cov()
        self.e1 = Ellipse(200, 100, radians(0))
        self.e2 = Ellipse(400, 20, radians(0))
        self.e1.o = Vec(400, 30, 0)
        self.e2.o = Vec(40, 180, 0)
        self.e3 = Ellipse(1, 1, 0)
        self.e4 = Ellipse(1, 1, 0)
        self.parameter = np.linspace(0, 2 * pi, 100)
        self.observer = None
        self.figure = ax
        # back to front order
        self.e3series = ax.line(x=[], y=[], color=(255, 200, 200))
        self.e1scaledseries = ax.line(x=[], y=[], color=(200, 200, 200))
        self.e2scaledseries = ax.line(x=[], y=[], color=(200, 200, 200))
        self.e4series = ax.line(x=[], y=[], color=(87, 87, 87))
        self.e1series = ax.line(x=[], y=[], color=self.mpl_palette[0], line_width=2)
        self.e2series = ax.line(x=[], y=[], color=self.mpl_palette[1], line_width=2)

    def compute(self):
        self.sep = sep = self.e2.o - self.e1.o
        e3 = self.e3
        lensep = self.sep.normalize()
        self.radius = radius = e3.radius(atan2(sep.dot(e3.dv), sep.dot(e3.du)))
        self.k = lensep / radius

    def make_ecombined(self) -> Ellipse:
        self.cov = combined_ellipse_to_cov(self.e1, self.e2)
        self.e3 = cov_to_ellipse(self.cov)
        self.e3.o = self.e1.o
        self.compute()
        return self.e3

    def _points(self, ell: Ellipse) -> tuple[list[float], list[float]]:
        x = [p.x for p in (ell.o + ell.parameterized(float(r)) for r in self.parameter)]
        y = [p.y for p in (ell.o + ell.parameterized(float(r)) for r in self.parameter)]
        return x, y

    def plot4(self):
        self.e4 = self.e3.scale(self.k)
        x, y = self._points(self.e4)
        self.redo(self.e4series, x, y)

    def plot3(self):
        e3_scaled = self.e3.scale(setup.k)
        x, y = self._points(e3_scaled)
        self.redo(self.e3series, x, y)

    def finalize_plot(self):
        self.make_ecombined()
        self.plot3()
        self.plot4()
        self.notify()

    def plot1(self, rot: float, solo: bool = False):
        self.e1.set_rotation(std_rot(rot))
        self.plot1_scaled(setup.k)
        x, y = self._points(self.e1)
        self.redo(self.e1series, x, y)
        if not solo:
            self.finalize_plot()

    def plot1_scaled(self, k: float):
        e1_scaled = self.e1.scale(k)
        x, y = self._points(e1_scaled)
        self.redo(self.e1scaledseries, x, y)

    def plot2(self, rot: float):
        self.e2.set_rotation(std_rot(rot))
        self.plot2_scaled(setup.k)
        x, y = self._points(self.e2)
        self.redo(self.e2series, x, y)
        self.finalize_plot()

    def plot2_scaled(self, k: float):
        e2_scaled = self.e2.scale(k)
        x, y = self._points(e2_scaled)
        self.redo(self.e2scaledseries, x, y)

    def redo(self, series, x, y):
        series.data_source.data = dict(x=x, y=y)

    def notify(self):
        if self.observer is not None:
            self.observer(self)

    def plot(self):
        self.make_ecombined()
        self.plot1(self.e1.rot, solo=True)
        self.plot2(self.e2.rot)

        # Center line and Labels
        cx = [self.e1.o.x, self.e2.o.x]
        cy = [self.e1.o.y, self.e2.o.y]
        self.figure.line(x=cx, y=cy, color="green")
        self.figure.text(x=cx, y=cy, text=["B", "A"])

    def perp(self):
        self.parallel(atan2(self.sep.y, self.sep.x) + pi / 2)

    def parallel(self, angle):
        self.plot1(angle, solo=True)
        self.plot2(angle)

    def rotate(self, angle):
        self.plot1(self.e1.rot + angle, solo=True)
        self.plot2(self.e2.rot + angle)

    def reasonable_limits(
        self, factor: float = 1.1
    ) -> tuple[float, float, float, float]:
        s = 0.0
        xmin, xmax = 1e12, -1e12
        ymin, ymax = 1e12, -1e12
        ell = self.e1, self.e2
        for e in ell:
            d = e.a + e.b
            s += d * d
        s = sqrt(s) * factor
        for e in ell:
            ox, oy = e.o.x, e.o.y
            if ox - s < xmin:
                xmin = ox - s
            if ox + s > xmax:
                xmax = ox + s
            if oy - s < ymin:
                ymin = oy - s
            if oy + s > ymax:
                ymax = oy + s
        return (xmin, xmax, ymin, ymax)

In [ ]:
from jupyter_bokeh import BokehModel
from bokeh.plotting import figure

fig = figure(
    sizing_mode="stretch_width",
    height=600,
    match_aspect=True,
)
fig.grid.visible = False
scaled = Scaled(fig)

updating = False


def update(s: Scaled):
    global updating
    try:
        updating = True
        ra.value = -degrees(s.e2.rot)
        rb.value = -degrees(s.e1.rot)
    finally:
        updating = False


def showconf():
    # Separation factor
    sep = scaled.e2.o - scaled.e1.o
    c2c = sep.normalize()
    pua = ellipse_to_cov(scaled.e1)
    apcr = pua.radius(sep)
    pub = ellipse_to_cov(scaled.e2)
    bpcr = pub.radius(sep)
    sf = c2c / (setup.k * hypot(apcr, bpcr))
    # P value
    p = sqrt(chi2.cdf(scaled.k * scaled.k, 3))
    conf_label.value = f"P-value: {p:.4f} Simple SF: {sf:.3f}"
    sf_values.value = (
        f"Apcr={apcr:.1f} Bpcr={bpcr:.1f} K={setup.k:.2f} rss={hypot(apcr, bpcr):.1f}"
    )
    pass


def set_conf_scale(v: float):
    setup.k = v
    scaled.plot1_scaled(setup.k)
    scaled.plot2_scaled(setup.k)
    scaled.plot3()
    showconf()


def on_rota(r):
    if updating:
        return
    scaled.plot2(-radians(r.new))


def on_rotb(r):
    if updating:
        return
    scaled.plot1(-radians(r.new))


ra = widgets.FloatSlider(
    description="A rot",
    min=-180,
    max=180,
    value=-degrees(std_rot(scaled.e2.rot)),
    step=1.0,
)
rb = widgets.FloatSlider(
    description="B rot",
    min=-180,
    max=180,
    value=-degrees(std_rot(scaled.e1.rot)),
    step=1.0,
)

ra.observe(on_rota, names="value")
rb.observe(on_rotb, names="value")

H = widgets.HBox
V = widgets.VBox

ccw15 = "15\u00b0 \u21ba"
cw15 = "15\u00b0 \u21bb"

conf_label = widgets.Label()
sf_values = widgets.Label()
conf_scale = widgets.FloatSlider(
    description="K", value=setup.k, min=1.0, max=4.0, step=0.01
)
halfwidth = widgets.Layout(width="50%")
button_align = widgets.Button(description="Align")
button_align.on_click(lambda b: scaled.perp())
button_90 = widgets.Button(description="90", layout=halfwidth)
button_90.on_click(lambda b: scaled.parallel(pi / 2))
button_0 = widgets.Button(description="0", layout=halfwidth)
button_0.on_click(lambda b: scaled.parallel(0))
button_more = widgets.Button(description=ccw15, layout=halfwidth)
button_more.on_click(lambda b: scaled.rotate(pi / 12))
button_less = widgets.Button(description=cw15, layout=halfwidth)
button_less.on_click(lambda b: scaled.rotate(-pi / 12))
button_bmore = widgets.Button(description=f"B {ccw15}", layout=halfwidth)
button_bmore.on_click(lambda b: scaled.plot1(scaled.e1.rot + pi / 12))
button_bless = widgets.Button(description=f"B {cw15}", layout=halfwidth)
button_bless.on_click(lambda b: scaled.plot1(scaled.e1.rot - pi / 12))
button_amore = widgets.Button(description=f"A {ccw15}", layout=halfwidth)
button_amore.on_click(lambda b: scaled.plot2(scaled.e2.rot + pi / 12))
button_aless = widgets.Button(description=f"A {cw15}", layout=halfwidth)
button_aless.on_click(lambda b: scaled.plot2(scaled.e2.rot - pi / 12))

ra.observe(lambda v: showconf(), names="value")
rb.observe(lambda v: showconf(), names="value")
conf_scale.observe(lambda v: set_conf_scale(v.new), names="value")

fig.match_aspect = True
fig.aspect_scale = 1.0

scaled.observer = update
scaled.plot()

In [ ]:
showconf()
display(
    V(
        [
            widgets.HTML("""<div style='font-size:smaller; background-color:rgb(from pink r g b/5%);padding:1em 0.5em'>
                         Before you play with the buttons, it's nicer if you click on the plot and zoom/pan first...
                         <i>(I can't get it work any better than this :sadface:)</i>
                         </div>""", layout=widgets.Layout(border="1px solid pink")),

            H(
                [
                    V(
                        [
                            button_align,
                            H([button_90, button_0]),
                            H([button_more, button_less]),
                        ]
                    ),
                    V([ra, conf_scale, H([button_amore, button_aless])]),
                    V([rb, widgets.Label(), H([button_bmore, button_bless])]),
                    V(
                        [
                            conf_label,
                            sf_values,
                        ]
                    ),
                ]
            ),
        ]
    ),
    BokehModel(fig),
)

Simple SF ACR= $ C \over {K \sqrt{{R_A}^2 + {R_B}^2}} $ where $C$ is distance from B to A and $R_A$ and $R_B$ are the pedal radii of A and B

Pale grey ellipses at A and B are scaled by $K$

Pink ellipse is A combined (randomly) with B scaled by $K$ - its pedal radius appears in the separation factor as $ \sqrt{{R_A}^2 + {R_B}^2} $

Dark grey ellipse is combined ellipse scaled to intersect A    
It is the probability "contour" on which A lies
